In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error, explained_variance_score
import math
import pandas as pd
from numpy import *
import pandas as pd
from sklearn.metrics import mean_squared_error, explained_variance_score
import time
import os

In [ ]:
#lssvm#
def predict(alphas,b,dataMat,testVec):
    Kx = kernelTrans(dataMat,testVec,kTup)   
    predict_value =  Kx.T * alphas + b
    return predict_value'

C = 0.6
k1 = 0.3
kernel = 'rbf'
kTup = (kernel,k1)


xlsbpath=r"/"
os.chdir(xlsbpath) 
filelist= os.listdir(xlsbpath)

def kernelTrans(X,A,kTup):
    X = mat(X)
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0] == 'lin':
        K = X * A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow * deltaRow.T
        K = exp(K/(-1 * kTup[1] ** 2))
    else: raise NameError('Houston We Have a Problem -- That Kernel is not recognized')
    return K   
class optStruct:
    def __init__(self,dataMatIn,classLabels,C,kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.K = mat(zeros((self.m,self.m)))  
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)
def leastSquares(dataMatIn,classLabels,C,kTup):
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,kTup)

    unit = mat(ones((oS.m,1)))  #[1,1,...,1].T
    I = eye(oS.m)
    zero = mat(zeros((1,1)))
    upmat = hstack((zero,unit.T))
    downmat = hstack((unit,oS.K + I/float(C)))

    completemat = vstack((upmat,downmat))  
    rightmat = vstack((zero,oS.labelMat))   
    b_alpha = completemat.I * rightmat
    oS.b = b_alpha[0,0]
    for i in range(oS.m):
        oS.alphas[i,0] = b_alpha[i+1,0]
    return oS.alphas,oS.b,oS.K

def predict(alphas,b,dataMat,testVec):
    Kx = kernelTrans(dataMat,testVec,kTup)   
    predict_value =  Kx.T * alphas + b
    return predict_value

In [ ]:
for k in range(len(filelist)):
    time1=time.time()
    df=pd.read_excel("/"+filelist[k])
    dataMat=[]
    labelMat=[]
    for i in range(len(df)):
        dataMat.append([df['lon'][i],df['lat'][i],df['hig'][i],int(df['time_'][i].split(' ')[0].split('/')[0]),
                        int(df['time_'][i].split(' ')[0].split('/')[1]),int(df['time_'][i].split(' ')[0].split('/')[2]),
                        int(df['time_'][i].split(' ')[1].split(':')[0]),df['hgpt_ztd'][i]])
        labelMat.append(df['ztd'][i]-df['hgpt_ztd'][i])
    alphas,b,K = leastSquares(dataMat,labelMat,C,kTup)
    pred_=[]
    for j in range(len(df)):
        pred=predict(alphas,b,dataMat,dataMat[j])
        pred_.append(pred)
#     pred_L.append(pred_)
    pred_L=[]
    for i in range(len(pred_)):
        pred_L.append(pred_[i].tolist()[0][0])
    time2=time.time()
    df.insert(df.shape[1],'diff_v',pred_L)
    Tem=[]
    for z in range(len(df)):
        tem=df['tcn_ztd'][z]+pred_L[z]
        Tem.append(tem)
    df.insert(df.shape[1],'tcn_lssvm_ztd',Tem)
    df.to_excel("/"+filelist[k],index=None)
    print(filelist[k]+"time:"+str(time2-time1)+"s，rest:"+str(34-k))

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LeakyReLU, Dropout, Input, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, explained_variance_score
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import math
scaler = MinMaxScaler()

pred_L=[]
for k in range(len(filelist)):
    try:
        df=pd.read_excel("/"+filelist[k])
        df=df.drop(['gpt3_ztd','tcn_ztd','tcn_lssvm_ztd'],axis=1)
        hgpt_diff=[]
        for m in range(len(df)):
            hgpt_diff.append(df['hgpt_ztd'][m]+df['diff_v'][m])
        df.insert(df.shape[1],'hgpt_diff',hgpt_diff)
        df=df.drop(['hgpt_ztd','diff_v'],axis=1)
        a=0 
        b=0
        c=0 
        d=0 
        e=0
        for j in range(len(df['time_'])): 
            if df['time_'][j][0:4]=='2018':
                a+=1
            elif df['time_'][j][0:4]=='2019':
                b+=1
            elif df['time_'][j][0:4]=='2020':
                c+=1
            elif df['time_'][j][0:4]=='2021':
                d+=1
            elif df['time_'][j][0:4]=='2022':
                e+=1
        df1=df[0:a]
        df2=df[a:a+b]
        df3=df[a+b:a+b+c]
        df4=df[a+b+c:a+b+c+d]
        df5=df[a+b+c+d:a+b+c+d+e]
        time=[df1,df2,df3,df4,df5]
        kernel_size = 2
        dropout = 0.2
        pred_=[]
        for i in range(len(time)):
            X = time[i].drop(["ztd","time_"], axis = 1)
            y = time[i]["ztd"]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_test = np.expand_dims(X_test, axis=2)
            model = Sequential()
            model.add(Conv1D(32, kernel_size, padding = "causal", input_shape = X_train.shape[1:]))
            model.add(LeakyReLU(alpha = 0.1))
            model.add(BatchNormalization())

            model.add(Conv1D(64, kernel_size, padding = "causal",  dilation_rate = 2))
            model.add(LeakyReLU(alpha = 0.01))
            model.add(BatchNormalization())

            model.add(Conv1D(128, kernel_size, padding = "causal", activation = "relu", dilation_rate = 4))
            model.add(BatchNormalization())

            model.add(Flatten())

            model.add(Dense(128, activation = "relu"))
            model.add(Dropout(dropout))
            model.add(Dense(1, activation = "relu"))
            filepath='weights.best.hdf5'
            checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,save_best_only=True,mode='min',period=1)
            callbacks_list = [checkpoint]
            model.compile(loss= "mean_squared_error", optimizer= "adam")
        #     model.fit(x=X_train, y=y_train, batch_size =50, epochs=300, validation_data=(X_test, y_test), verbose=1)
            model.fit(x=X_train, y=y_train, batch_size =50,epochs=300,validation_data=(X_test, y_test),verbose=1,callbacks=callbacks_list)
            model=load_model('weights.best.hdf5')


            X = scaler.transform(X)
            X = np.expand_dims(X, axis=2)
            pred=model.predict(X)
            pred_.append(pred)
        pred_L.append(pred_)
        print(k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k,k)
    except:
        print(k)